In [9]:
# A simple pipeline structure that can run different steps in a compute cluster
import azure.core
from azureml.core import Workspace, Datastore, Environment, Experiment, ScriptRunConfig
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline
from azureml.core.runconfig import RunConfiguration
from azureml.widgets import RunDetails
from azureml.data import OutputFileDatasetConfig
import json

ws = Workspace.from_config()

blob_store = Datastore(ws, "workspaceblobstore")    # workspaceblobstore is the default

# Create Experiment
experiment_name = 'train_merge'
exp = Experiment(ws, experiment_name)

# Create Compute Target
aml_compute_target = 'vdsm-cluster-l80'  # ALL SMALL LETTER, NO underscore, 16ws long only.
aml_compute = AmlCompute(ws, aml_compute_target)

# Use an existing env which was registered to the ws
env = Environment.get(workspace=ws,name="vdsm_all",version="1")

In [ ]:
# A simple pipeline structure that can run different steps in a compute cluster
from azureml.pipeline.steps import PythonScriptStep, ParallelRunStep, ParallelRunConfig
from azureml.pipeline.core.pipeline_output_dataset import PipelineOutputTabularDataset
from azureml.data.dataset_consumption_config import DatasetConsumptionConfig
from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.core.graph import PipelineParameter

pipeline_run_config = ParallelRunConfig(environment=env,
                                        entry_script="train_merge.py",
                                        error_threshold=-1,
                                        output_action="summary_only",
                                        compute_target=aml_compute,
                                        process_count_per_node=20,
                                        run_invocation_timeout=10800,
                                        node_count=2)

start_month_pipeline_param = PipelineParameter(name="start_month", default_value="2015-11")
end_month_pipeline_param = PipelineParameter(name="end_month", default_value="2021-07")

ds_list = []
for i in range(1,249,2):
    ds_list.append(DatasetConsumptionConfig(name="row_{}_{}".format(i, i+1), dataset=ws.datasets.get("row_{}_{}".format(i, i+1))))

ds_list.append(DatasetConsumptionConfig(name="row_249", dataset=ws.datasets.get("row_249")))

dataprep_step = ParallelRunStep(
    name="merge_for_training",
    inputs=ds_list,
    arguments=[
        "--start_month", start_month_pipeline_param,
        "--end_month", end_month_pipeline_param,
    ],
    parallel_run_config=pipeline_run_config)

steps = [dataprep_step]

pipeline = Pipeline(workspace=ws, steps=steps)
pipeline_run = exp.submit(pipeline)
print('VDSM training data fusion submitted for execution')
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion()